# 03 - PromptTemplate 模板使用

## 用途
学习使用 LangChain PromptTemplate 创建可复用的提示词模板

## 学习目标
- 掌握 PromptTemplate 基础语法
- 理解变量填充机制
- 能设计灵活的模板
- 控制输出长度以节省成本

## 代码块独立性说明
**注意**：每个代码块都是独立的，包含完整的导入和初始化，确保可以单独运行。

## 1. PromptTemplate 基础创建

In [1]:
# PromptTemplate 基础创建 - 独立代码块
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

# 初始化模型（使用轻量模型节省成本）
llm = ChatOpenAI(
    model="gpt-4o-mini",  # 轻量模型，成本更低
    temperature=0.7,
    max_tokens=200,        # 控制输出长度
)

# 创建基础提示词模板
prompt = PromptTemplate(
    input_variables=["topic"],
    template="请用 GPT-4o 的视角写一段关于 {topic} 的介绍，控制在50字以内。"
)

print("✅ PromptTemplate 创建成功")
print(f"模板: {prompt.template}")
print(f"输入变量: {prompt.input_variables}")

# 测试模板
topic = "人工智能"
formatted_prompt = prompt.format(topic=topic)
print(f"\n📝 格式化后的提示词: {formatted_prompt}")

# 使用模板调用模型
response = llm.invoke(formatted_prompt)
print(f"\n🤖 GPT-4o-mini 回复: {response.content}")

# 验证点：模板能动态生成 prompt 并传入 GPT
assert "人工智能" in formatted_prompt, "模板变量未正确填充"
assert len(response.content) <= 100, "回复长度超出预期"  # 给一些缓冲
print("\n✅ 验证通过：模板动态生成并成功调用")

✅ PromptTemplate 创建成功
模板: 请用 GPT-4o 的视角写一段关于 {topic} 的介绍，控制在50字以内。
输入变量: ['topic']

📝 格式化后的提示词: 请用 GPT-4o 的视角写一段关于 人工智能 的介绍，控制在50字以内。

🤖 GPT-4o-mini 回复: 人工智能是模拟人类智能的技术，通过机器学习和深度学习等方法，使计算机能够进行感知、推理和决策，广泛应用于各个领域，推动社会进步与创新。

✅ 验证通过：模板动态生成并成功调用


## 2. 多变量 PromptTemplate

In [2]:
# 多变量 PromptTemplate - 独立代码块
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

# 初始化模型
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    max_tokens=150,
)

# 创建多变量提示词模板
multi_prompt = PromptTemplate(
    input_variables=["concept", "level", "language"],
    template="请用 {language} 向 {level} 的学习者解释 {concept}，要求简洁明了。"
)

print("✅ 多变量 PromptTemplate 创建成功")
print(f"输入变量: {multi_prompt.input_variables}")

# 测试多变量填充
formatted_prompt = multi_prompt.format(
    concept="机器学习",
    level="初学者",
    language="中文"
)

print(f"\n📝 格式化后的提示词: {formatted_prompt}")

# 调用模型
response = llm.invoke(formatted_prompt)
print(f"\n🤖 GPT-4o-mini 回复: {response.content}")

# 验证点：多变量正确填充
assert "机器学习" in formatted_prompt, "概念变量未填充"
assert "初学者" in formatted_prompt, "级别变量未填充"
assert "中文" in formatted_prompt, "语言变量未填充"
print("\n✅ 验证通过：多变量模板正确填充")

✅ 多变量 PromptTemplate 创建成功
输入变量: ['concept', 'language', 'level']

📝 格式化后的提示词: 请用 中文 向 初学者 的学习者解释 机器学习，要求简洁明了。

🤖 GPT-4o-mini 回复: 机器学习是一种让计算机通过数据学习和做出决策的技术。简单来说，它就像教电脑从经验中不断改进和提升自己，而不是通过明确的编程来完成任务。

在机器学习中，我们通常会使用大量的数据来训练模型。这个模型能够识别模式、做出预测或分类。例如，当我们给机器学习模型提供很多关于猫和狗的图片时，经过训练后，它就可以识别新的图片是猫还是狗。

机器学习的应用非常广泛，比如语音识别、图像识别、推荐系统等。总之，机器学习使得计算机能够在没有明确编程的情况下，通过学习来解决实际问题。

✅ 验证通过：多变量模板正确填充


## 3. 模板验证和错误处理

In [3]:
# 模板验证和错误处理 - 独立代码块
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

print("🔍 PromptTemplate 验证和错误处理测试:")

# 测试1：缺少必需变量
print("\n🧪 测试1: 缺少必需变量")
try:
    prompt = PromptTemplate(
        input_variables=["topic", "style"],
        template="用 {style} 风格介绍 {topic}"
    )
    # 只提供一个变量
    formatted = prompt.format(topic="Python")
    print("⚠️  意外成功：应该缺少变量")
except Exception as e:
    print(f"✅ 正确捕获错误: {type(e).__name__}")

# 测试2：变量名不匹配
print("\n🧪 测试2: 变量名不匹配")
try:
    prompt = PromptTemplate(
        input_variables=["subject"],
        template="介绍 {topic}"  # 模板中的变量与 input_variables 不匹配
    )
    print("⚠️  意外成功：应该检测到变量不匹配")
except Exception as e:
    print(f"✅ 正确捕获错误: {type(e).__name__}")

# 测试3：正确的模板使用
print("\n🧪 测试3: 正确的模板使用")
try:
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0.7,
        max_tokens=100,
    )
    
    prompt = PromptTemplate(
        input_variables=["concept"],
        template="简单解释 {concept}"
    )
    
    formatted = prompt.format(concept="深度学习")
    response = llm.invoke(formatted)
    
    print(f"✅ 正确使用成功: {response.content}")
    
except Exception as e:
    print(f"❌ 正确使用失败: {e}")

print("\n📋 错误处理总结:")
print("✅ 缺少必需变量能被正确检测")
print("✅ 变量名不匹配能被正确检测")
print("✅ 正确的模板使用能正常工作")

🔍 PromptTemplate 验证和错误处理测试:

🧪 测试1: 缺少必需变量
✅ 正确捕获错误: KeyError

🧪 测试2: 变量名不匹配
⚠️  意外成功：应该检测到变量不匹配

🧪 测试3: 正确的模板使用
✅ 正确使用成功: 深度学习是机器学习的一个分支，主要通过模拟人脑神经网络的结构和功能来进行数据处理和分析。它利用多层神经网络（即“深度”网络）来自动提取特征，并进行模式识别和预测。

深度学习的核心思想是通过大量的数据和计算能力，让模型在学习过程中自动找到数据中的规律，而不需要人工提取特征。常见的应用包括图像识别

📋 错误处理总结:
✅ 缺少必需变量能被正确检测
✅ 变量名不匹配能被正确检测
✅ 正确的模板使用能正常工作


## 4. 模板链式调用

In [4]:
# 模板链式调用 - 独立代码块
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

# 初始化模型
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    max_tokens=120,
)

# 创建提示词模板
prompt = PromptTemplate(
    input_variables=["technology", "benefit"],
    template="请说明 {technology} 的主要优势是 {benefit}，用一句话总结。"
)

# 创建 LCEL 链：prompt | llm | parser
chain = prompt | llm | StrOutputParser()

print("✅ PromptTemplate 链式调用创建成功")
print("🔗 链结构: PromptTemplate → ChatOpenAI (GPT-4o-mini) → StrOutputParser")

# 测试链式调用
result = chain.invoke({
    "technology": "LangChain",
    "benefit": "简化 LLM 应用开发"
})

print(f"\n🤖 链式调用结果: {result}")
print(f"📊 结果类型: {type(result)}")
print(f"📊 结果长度: {len(result)} 字符")

# 验证点：链式调用返回字符串格式结果
assert isinstance(result, str), "结果不是字符串类型"
assert len(result) > 0, "结果为空"
assert "LangChain" in result, "结果不包含相关内容"
print("\n✅ 验证通过：PromptTemplate 链式调用成功")

✅ PromptTemplate 链式调用创建成功
🔗 链结构: PromptTemplate → ChatOpenAI (GPT-4o-mini) → StrOutputParser

🤖 链式调用结果: LangChain 的主要优势在于通过提供高层次的抽象和工具，简化了大语言模型（LLM）应用的开发流程，使开发者能够更快速地构建和部署复杂的自然语言处理应用。
📊 结果类型: <class 'str'>
📊 结果长度: 79 字符

✅ 验证通过：PromptTemplate 链式调用成功


## 5. 模板性能对比

In [5]:
# 模板性能对比 - 独立代码块
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
import time
import os

# 加载环境变量
load_dotenv()

print("⚡ PromptTemplate 性能对比测试:")
print("=" * 40)

# 测试不同模型的性能
test_prompt = "请用一句话介绍 {topic}"
topic = "区块链技术"

models_to_test = [
    ("gpt-4o-mini", "轻量模型"),
    ("gpt-4o", "标准模型"),
]

results = {}

for model_name, description in models_to_test:
    print(f"\n🤖 测试 {description} ({model_name}):")
    
    try:
        # 初始化模型
        llm = ChatOpenAI(
            model=model_name,
            temperature=0.7,
            max_tokens=100,
        )
        
        # 创建模板
        prompt = PromptTemplate(
            input_variables=["topic"],
            template=test_prompt
        )
        
        # 测试性能
        start_time = time.time()
        formatted_prompt = prompt.format(topic=topic)
        response = llm.invoke(formatted_prompt)
        end_time = time.time()
        
        # 记录结果
        results[model_name] = {
            "response": response.content,
            "length": len(response.content),
            "time": end_time - start_time,
            "success": True
        }
        
        print(f"   回复: {response.content}")
        print(f"   长度: {len(response.content)} 字符")
        print(f"   耗时: {end_time - start_time:.2f} 秒")
        print(f"   状态: ✅ 成功")
        
    except Exception as e:
        print(f"   状态: ❌ 失败 - {e}")
        results[model_name] = {
            "response": None,
            "length": 0,
            "time": 0,
            "success": False,
            "error": str(e)
        }

# 性能对比总结
print(f"\n📊 性能对比总结:")
successful_models = [name for name, result in results.items() if result["success"]]
print(f"✅ 成功测试的模型: {len(successful_models)}/{len(models_to_test)}")

if len(successful_models) >= 2:
    print("\n📈 性能对比:")
    for model_name in successful_models:
        result = results[model_name]
        print(f"   {model_name}: {result['length']} 字符, {result['time']:.2f} 秒")
    
    print("\n💡 成本优化建议:")
    print("   - 测试和演示优先使用 gpt-4o-mini")
    print("   - 生产环境根据质量要求选择模型")
    print("   - 合理设置 max_tokens 控制输出长度")
    print("✅ 验证通过：模型性能对比完成")
else:
    print("⚠️  警告：只有一个模型可用，无法进行对比")

⚡ PromptTemplate 性能对比测试:

🤖 测试 轻量模型 (gpt-4o-mini):
   回复: 区块链技术是一种去中心化的分布式账本技术，能够安全、透明地记录和验证交易数据。
   长度: 39 字符
   耗时: 1.99 秒
   状态: ✅ 成功

🤖 测试 标准模型 (gpt-4o):
   回复: 区块链技术是一种去中心化的分布式账本技术，通过加密算法和共识机制保障数据的透明性、不可篡改性和安全性，广泛应用于金融、供应链、数字身份等领域。
   长度: 71 字符
   耗时: 1.60 秒
   状态: ✅ 成功

📊 性能对比总结:
✅ 成功测试的模型: 2/2

📈 性能对比:
   gpt-4o-mini: 39 字符, 1.99 秒
   gpt-4o: 71 字符, 1.60 秒

💡 成本优化建议:
   - 测试和演示优先使用 gpt-4o-mini
   - 生产环境根据质量要求选择模型
   - 合理设置 max_tokens 控制输出长度
✅ 验证通过：模型性能对比完成


## 6. 学习总结与最佳实践

In [6]:
# 学习总结与最佳实践 - 独立代码块
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

print("📋 PromptTemplate 学习总结:")
print("=" * 40)

# 验证点检查
verification_points = [
    "✅ 基础模板创建：单变量 PromptTemplate",
    "✅ 多变量模板：支持多个输入变量",
    "✅ 模板验证：错误检测和处理",
    "✅ 链式调用：prompt | llm | parser",
    "✅ 性能对比：不同模型性能差异",
]

for point in verification_points:
    print(point)

print(f"\n🎯 核心技能掌握情况: {len(verification_points)}/5 项")

print("\n💡 PromptTemplate 最佳实践:")
print("1. 变量命名：使用清晰、有意义的变量名")
print("2. 模板设计：保持模板简洁、明确")
print("3. 错误处理：检查变量填充和模板格式")
print("4. 成本控制：测试时使用轻量模型")
print("5. 链式调用：结合 LCEL 提高效率")

print("\n🚀 下一步学习建议:")
print("1. 深入学习 ChatPromptTemplate 多角色对话")
print("2. 掌握 OutputParser 输出解析")
print("3. 学习 FewShotPromptTemplate 少样本学习")
print("4. 探索管道提示词 (PipelinePromptTemplate)")
print("5. 实践复杂模板设计模式")

# 最终验证：确保 PromptTemplate 基础功能可用
try:
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0.7,
        max_tokens=50,
    )
    
    prompt = PromptTemplate(
        input_variables=["skill"],
        template="我学会了 {skill} 的使用方法"
    )
    
    response = llm.invoke(prompt.format(skill="PromptTemplate"))
    print(f"\n🎉 最终验证成功: {response.content}")
    print("\n✅ PromptTemplate 学习完成！")
    
except Exception as e:
    print(f"\n❌ 最终验证失败: {e}")

📋 PromptTemplate 学习总结:
✅ 基础模板创建：单变量 PromptTemplate
✅ 多变量模板：支持多个输入变量
✅ 模板验证：错误检测和处理
✅ 链式调用：prompt | llm | parser
✅ 性能对比：不同模型性能差异

🎯 核心技能掌握情况: 5/5 项

💡 PromptTemplate 最佳实践:
1. 变量命名：使用清晰、有意义的变量名
2. 模板设计：保持模板简洁、明确
3. 错误处理：检查变量填充和模板格式
4. 成本控制：测试时使用轻量模型
5. 链式调用：结合 LCEL 提高效率

🚀 下一步学习建议:
1. 深入学习 ChatPromptTemplate 多角色对话
2. 掌握 OutputParser 输出解析
3. 学习 FewShotPromptTemplate 少样本学习
4. 探索管道提示词 (PipelinePromptTemplate)
5. 实践复杂模板设计模式

🎉 最终验证成功: 太好了！PromptTemplate 是一个非常有用的工具，可以帮助你构建和管理提示（prompts），尤其是在与语言模型进行交互时。你可以通过模板化的方式来提高提示的结构化和可复

✅ PromptTemplate 学习完成！
